In [1]:
import numpy as np

# Feed forward

## Data Matrices

For the fully connected feed forward neural network, we have multiple important matrices to keep track of. Let's first define the data matrices for the undefined supervised learning problem that our network will be training on. 

$$\underline{X} = \begin{bmatrix}
    x_{1,1} & \cdots & x_{1,m} \\
    \vdots & \ddots & \vdots \\
    x_{n_x,1} & \cdots & x_{n_x,m}
\end{bmatrix}$$

Here, $n_x$ is the number of features in the input dataset. $m$ is the number of training examples we will feed into the network. So $\underline{X}$ can be thought of a collection of column vectors where each one represents a datapoint.

$\underline{Y}$ is defined similarly, where this is the matrix that contains the labels for the training data. 

$$\underline{Y} = \begin{bmatrix}
    y_1 & \cdots & y_i \cdots & y_m
\end{bmatrix}$$

$y_i$ is intentionally left ambiguous here, as it will vary depending on the problem. If it is regression or binary classification, it will be a number, if it is multiclass classification, it will be a vector.

In [9]:
# For now, invent meaningless data that is already normalised for development.
X = np.random.randn(100, 1000) # 100 features, 1000 examples
Y = np.random.randint(2, size=(1,1000)) # the training labels for binary classification for 1000 examples
nx = X.shape[0]
m = X.shape[1]
ny = Y.shape[0]
print("The number of features are ", nx)
print("The number of training examples are ", m)

The number of features are  100
The number of training examples are  1000
